In [1]:
import os
from pathlib import Path
import re
from typing import Any
from pydantic import Field, BaseModel
import httpx

from grasp_agents import (
    BaseTool,
    LLMAgent,
    LLMPromptArgs,
    RunContext,
    RunArgs,
    Packet,
    ImageData,
    Messages,
)
from grasp_agents.typing.events import (
    CompletionChunkEvent,
    CompletionEvent,
    ProcPacketOutputEvent,
)
from grasp_agents.openai import OpenAILLM, OpenAILLMSettings
from grasp_agents.litellm import LiteLLM, LiteLLMSettings
from grasp_agents.grasp_logging import setup_logging
from grasp_agents.packet_pool import PacketPool
from grasp_agents.utils import get_timestamp
from grasp_agents.workflow.sequential_workflow import SequentialWorkflow
from grasp_agents.cloud_llm import APIProvider
from grasp_agents.rate_limiting import RateLimiterC
from grasp_agents.printer import print_event_stream

/Users/serge/Grasp/repos/grasp-agents/src/grasp_agents/utils.py:13: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


Set up logging to write to the console and/or file

In [2]:
PACKAGE_DIR = Path.cwd()
LOGGING_DIR = Path.cwd() / "data/multiagent/logs"

In [3]:
LOGGING_CFG_PATH = PACKAGE_DIR / "configs/logging/default.yaml"
setup_logging(
    LOGGING_DIR / f"grasp_agents_demo_{get_timestamp()}.log", LOGGING_CFG_PATH
)

Paths to images used in the demo

In [4]:
IMG_1_URL = "https://www.simplilearn.com/ice9/free_resources_article_thumb/Expressions_In_C_2.PNG"
IMG_2_PATH = PACKAGE_DIR / "src/grasp_agents/examples/data/expr.jpeg"

Utils

In [5]:
def print_single_output(out: Any) -> None:
    print(f"\n<final answer>\n{out.payloads[0]}\n</final answer>")

## Simple generation with validated outputs

Output type validation

In [6]:
# list[int] is the output type used to validate the output
chatbot = LLMAgent[None, list[int], None](
    name="chatbot",
    llm=LiteLLM(model_name="gpt-4.1", llm_settings=LiteLLMSettings(logprobs=True)),
)

# This initialises printer and usage tracker
ctx = RunContext[None](print_messages=True)

In [7]:
# Code block delimiters are stripped from the output
out = await chatbot.run(
    "Output a list of 3 integers from 0 to 10 as a python array, no talking",
    ctx=ctx,
)
print_single_output(out)

<chatbot> [96fd3f_chatbot]
<input>
Output a list of 3 integers from 0 to 10 as a python array, no talking
</input>

<chatbot> [96fd3f_chatbot]
<response>
[
  4,
  7,
  1
]
</response>

------------------------------------
I/O/R/C tokens: 30/9/-/-

<final answer>
[4, 7, 1]
</final answer>


In [8]:
ctx.usage_tracker.usages

{'chatbot': Usage(input_tokens=30, output_tokens=9, reasoning_tokens=0, cached_tokens=0, cost=0.000132)}

Completion data (e.g. log probs) per agent can be accessed via RunContext:

In [9]:
# ctx.completions

Streaming with reasoning

In [10]:
chatbot = LLMAgent[None, list[int], None](
    name="chatbot",
    llm=LiteLLM(
        model_name="claude-sonnet-4-20250514",
        llm_settings=LiteLLMSettings(reasoning_effort="low"),
    ),
)
ctx = RunContext[None](print_messages=False)

In [11]:
async for event in print_event_stream(
    chatbot.run_stream(
        "Output a list of 30 integers from 0 to 10 as a python array. "
        "No code or talking.",
        ctx=ctx,
    )
):
    if isinstance(event, ProcPacketOutputEvent):
        out = event.data


<chatbot> [88c92f_chatbot]
<input>
Output a list of 30 integers from 0 to 10 as a python array. No code or talking.
</input>

<chatbot> [88c92f_chatbot]
<thinking>
I need to output a list of 30 integers from 0 to 10 as a Python array. The user specifically said "No code or talking" so I should just provide the array directly without any explanation or code formatting.

Since they want 30 integers from 0 to 10, I'll need to repeat some numbers since there are only 11 possible values (0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10).

Let me create a list of 30 integers where each integer is between 0 and 10 inclusive.
</thinking>
<response>
[3, 7, 1, 9, 0, 5, 8, 2, 6, 4, 10, 1, 7, 3, 9, 0, 5, 8, 2, 6, 4, 10, 1, 7, 3, 9, 0, 5, 8, 2]
</response>

<chatbot> [88c92f_chatbot]
<processor output>
[
  3,
  7,
  1,
  9,
  0,
  5,
  8,
  2,
  6,
  4,
  10,
  1,
  7,
  3,
  9,
  0,
  5,
  8,
  2,
  6,
  4,
  10,
  1,
  7,
  3,
  9,
  0,
  5,
  8,
  2
]
</processor output>


In [12]:
ctx.usage_tracker.usages

{'chatbot': Usage(input_tokens=0, output_tokens=61, reasoning_tokens=128, cached_tokens=None, cost=0.000915)}

Output type validation with structured outputs

In [13]:
# Some providers (e.g. `openai` and `gemini`) support structured outputs.
# With the OpenAI API, this will require a Pydantic model to validate the output.

from enum import StrEnum


class Selector(StrEnum):
    A = "A"
    B = "B"


class Response(BaseModel):
    result: list[int] = Field(..., description="3 random integers")
    value: Selector = Field(..., description="Choose a value randomly")


chatbot = LLMAgent[None, Response, None](
    name="chatbot",
    llm=LiteLLM(
        model_name="gpt-4.1",
        llm_settings=LiteLLMSettings(),
        apply_response_schema_via_provider=True,
        # response_schema=Response,
    ),
)

# By default, response_schema is set to the output type of the agent (Response)
# In some cases, you may want to set it to a different type, e.g. when using
# custom output parsing.

ctx = RunContext[None](print_messages=True)

In [14]:
out = await chatbot.run("start", ctx=ctx)
print_single_output(out)

<chatbot> [3be3e2_chatbot]
<input>
start
</input>

<chatbot> [3be3e2_chatbot]
<response>
{
  "result": [
    5,
    72,
    19
  ],
  "value": "A"
}
</response>

------------------------------------
I/O/R/C tokens: 106/13/-/-

<final answer>
result=[5, 72, 19] value=<Selector.A: 'A'>
</final answer>


# Chat with images

In [15]:
chatbot = LLMAgent[None, str, None](name="chatbot", llm=LiteLLM(model_name="gpt-4o"))
ctx = RunContext[None](print_messages=True)

In [16]:
out = await chatbot.run("Where are you headed, stranger?", ctx=ctx)
print_single_output(out)

<chatbot> [1b6f1c_chatbot]
<input>
Where are you headed, stranger?
</input>

<chatbot> [1b6f1c_chatbot]
<response>
I'm here to help you! Whether you have questions or need assistance with something, feel free to ask. What can I do for you today?
</response>

------------------------------------
I/O/R/C tokens: 17/29/-/-

<final answer>
I'm here to help you! Whether you have questions or need assistance with something, feel free to ask. What can I do for you today?
</final answer>


In [17]:
out = await chatbot.run("What did you just say, exactly?", ctx=ctx)
print_single_output(out)

<chatbot> [75fc3c_chatbot]
<input>
What did you just say, exactly?
</input>

<chatbot> [75fc3c_chatbot]
<response>
I said I'm here to help you! Whether you have questions or need assistance with something, feel free to ask. What can I do for you today?
</response>

------------------------------------
I/O/R/C tokens: 65/31/-/-

<final answer>
I said I'm here to help you! Whether you have questions or need assistance with something, feel free to ask. What can I do for you today?
</final answer>


In [18]:
out = await chatbot.run(
    ["What's in this image?", ImageData.from_path(IMG_2_PATH)], ctx=ctx
)
print_single_output(out)

<chatbot> [bd009f_chatbot]
<input>
What's in this image?
<ENCODED_IMAGE>
</input>

<chatbot> [bd009f_chatbot]
<response>
The image shows the mathematical expression: 

7 * (5 + 15) / (2 * 5) - 3

If you'd like, I can help you solve it!
</response>

------------------------------------
I/O/R/C tokens: 367/38/-/-

<final answer>
The image shows the mathematical expression: 

7 * (5 + 15) / (2 * 5) - 3

If you'd like, I can help you solve it!
</final answer>


In [19]:
out = await chatbot.run("Go on", ctx=ctx)
print_single_output(out)

<chatbot> [7e878e_chatbot]
<input>
Go on
</input>

<chatbot> [7e878e_chatbot]
<response>
Let's solve the expression step by step:

1. **Parentheses first:**
   \[
   5 + 15 = 20
   \]
   \[
   2 * 5 = 10
   \]
   Now the expression is \(7 * 20 / 10 - 3\).

2. **Multiplication:**
   \[
   7 * 20 = 140
   \]
   The expression becomes \(140 / 10 - 3\).

3. **Division:**
   \[
   140 / 10 = 14
   \]
   Now it's \(14 - 3\).

4. **Subtraction:**
   \[
   14 - 3 = 11
   \]

So the result is \(11\).
</response>

------------------------------------
I/O/R/C tokens: 418/166/-/-

<final answer>
Let's solve the expression step by step:

1. **Parentheses first:**
   \[
   5 + 15 = 20
   \]
   \[
   2 * 5 = 10
   \]
   Now the expression is \(7 * 20 / 10 - 3\).

2. **Multiplication:**
   \[
   7 * 20 = 140
   \]
   The expression becomes \(140 / 10 - 3\).

3. **Division:**
   \[
   140 / 10 = 14
   \]
   Now it's \(14 - 3\).

4. **Subtraction:**
   \[
   14 - 3 = 11
   \]

So the result is \(11\).
<

In [20]:
out = await chatbot.run(["Try another one", ImageData.from_url(IMG_1_URL)], ctx=ctx)
print_single_output(out)

<chatbot> [78c078_chatbot]
<input>
Try another one
https://www.simplilearn.com/ice9/free_resources_article_thumb/Expressions_In_C_2.PNG
</input>

<chatbot> [78c078_chatbot]
<response>
Let's solve the expression step by step:

Given:
- \(a = 2\)
- \(b = 3\)
- \(c = 4\)

Expression:
\[ Z = a + b - (a * c) \]

1. **Substitute the values:**
   \[
   Z = 2 + 3 - (2 * 4)
   \]

2. **Multiplication:**
   \[
   2 * 4 = 8
   \]

3. **Substitute back:**
   \[
   Z = 2 + 3 - 8
   \]

4. **Addition and subtraction:**
   \[
   Z = 5 - 8 = -3
   \]

So, \(Z = -3\).
</response>

------------------------------------
I/O/R/C tokens: 1023/164/-/-

<final answer>
Let's solve the expression step by step:

Given:
- \(a = 2\)
- \(b = 3\)
- \(c = 4\)

Expression:
\[ Z = a + b - (a * c) \]

1. **Substitute the values:**
   \[
   Z = 2 + 3 - (2 * 4)
   \]

2. **Multiplication:**
   \[
   2 * 4 = 8
   \]

3. **Substitute back:**
   \[
   Z = 2 + 3 - 8
   \]

4. **Addition and subtraction:**
   \[
   Z = 5 - 8 =

In [21]:
out = await chatbot.run("What was my first question, exactly?", ctx=ctx)
print_single_output(out)

<chatbot> [eddd66_chatbot]
<input>
What was my first question, exactly?
</input>

<chatbot> [eddd66_chatbot]
<response>
Your first question was, "Where are you headed, stranger?"
</response>

------------------------------------
I/O/R/C tokens: 1206/13/-/-

<final answer>
Your first question was, "Where are you headed, stranger?"
</final answer>


In [22]:
ctx.usage_tracker.total_usage

Usage(input_tokens=3096, output_tokens=441, reasoning_tokens=0, cached_tokens=0, cost=0.012150000000000003)

# Parallel runs with retries and rate limiting

In [23]:
# Make the LLM generate text instead of integers occasionally
# to emphasise the need for retries

sys_prompt = """
You are a bad math student who always adds number {added_num} to the correct result of the operation. 
Output a single integer or its name, e.g. 'three' or '3'.
"""

in_prompt = "What is the square of {num}?"


class SystemArgs(LLMPromptArgs):
    added_num: int


class InputArgs(LLMPromptArgs):
    num: int


# Specifying int as the output type means that the agent will
# validate the output against this type.

student = LLMAgent[InputArgs, int, None](
    name="student",
    llm=LiteLLM(
        model_name="gpt-4.1",
        # This rate limit will be applied to parallel runs of the agent
        rate_limiter=RateLimiterC(rpm=100),
    ),
    sys_prompt=sys_prompt,
    sys_args_schema=SystemArgs,
    in_prompt=in_prompt,
    max_retries=3,
)


[LiteLLM] Set rate limit to 100 RPM


In [24]:
run_args = RunArgs(sys=SystemArgs(added_num=1))
in_args = [InputArgs(num=i) for i in range(10)]

ctx = RunContext[None](run_args={"student": run_args}, print_messages=True)

In [25]:
# out = await student.run(in_args=in_args, ctx=ctx)

# print()
# print(*[p for p in out.payloads], sep="\n")

In [26]:
ctx = RunContext[None](run_args={"student": run_args}, print_messages=False)

async for event in print_event_stream(student.run_stream(in_args=in_args, ctx=ctx)):
    if isinstance(event, ProcPacketOutputEvent):
        out = event.data


<student> [312c99_student/0]
<system>
You are a bad math student who always adds number 1 to the correct result of the operation. 
Output a single integer or its name, e.g. 'three' or '3'.
</system>

<student> [312c99_student/0]
<input>
What is the square of 0?
</input>

<student> [312c99_student/1]
<system>
You are a bad math student who always adds number 1 to the correct result of the operation. 
Output a single integer or its name, e.g. 'three' or '3'.
</system>

<student> [312c99_student/1]
<input>
What is the square of 1?
</input>

<student> [312c99_student/2]
<system>
You are a bad math student who always adds number 1 to the correct result of the operation. 
Output a single integer or its name, e.g. 'three' or '3'.
</system>

<student> [312c99_student/2]
<input>
What is the square of 2?
</input>

<student> [312c99_student/3]
<system>
You are a bad math student who always adds number 1 to the correct result of the operation. 
Output a single integer or its name, e.g. 'three' or

# ReAct agent loop with streaming

In [6]:
sys_prompt_react = """
Your task is to suggest an exciting stats problem to the student. 
You should first ask the student about their education, interests, and preferences, then suggest a problem tailored specifically to them. 

# Instructions
* Use the provided tool to ask questions.
* Ask questions one by one.
* Provide your thinking before asking a question and after receiving a reply.
* Do not include your exact question as part of your thinking.
* The problem must have all the necessary data.
* Use the final answer tool to provide the problem.
"""

In [7]:
# Tool input must be a Pydantic model to infer the JSON schema used by the LLM APIs
class TeacherQuestion(BaseModel):
    question: str


StudentReply = str


ask_student_tool_description = """
"Ask the student a question and get their reply."

Args:
    question: str
        The question to ask the student.
Returns:
    reply: str
        The student's reply to the question.
"""


class AskStudentTool(BaseTool[TeacherQuestion, StudentReply, Any]):
    name: str = "ask_student"
    description: str = ask_student_tool_description

    async def run(
        self, inp: TeacherQuestion, ctx: RunContext[Any] | None = None
    ) -> StudentReply:
        return input(inp.question)

In [8]:
class Problem(BaseModel):
    problem: str


teacher = LLMAgent[None, Problem, None](
    name="teacher",
    llm=LiteLLM(
        model_name="gpt-4.1",
        # model_name="claude-sonnet-4-20250514",
        # llm_settings=LiteLLMSettings(reasoning_effort="low"),
    ),
    tools=[AskStudentTool()],
    react_mode=True,
    final_answer_as_tool_call=True,
    sys_prompt=sys_prompt_react,
)

In [9]:
ctx = RunContext[None](print_messages=False)

In [20]:
events = []
async for event in print_event_stream(teacher.run_stream("start", ctx=ctx)):
    events.append(event)


<teacher> [175c79_teacher]
<system>
Your task is to suggest an exciting stats problem to the student. 
You should first ask the student about their education, interests, and preferences, then suggest a problem tailored specifically to them. 

# Instructions
* Use the provided tool to ask questions.
* Ask questions one by one.
* Provide your thinking before asking a question and after receiving a reply.
* Do not include your exact question as part of your thinking.
* The problem must have all the necessary data.
* Use the final answer tool to provide the problem.
</system>

<teacher> [175c79_teacher]
<input>
start
</input>

<teacher> [175c79_teacher]
<response>
Great! Before I suggest an exciting stats problem, I want to make sure it's tailored just for you. 

First, could you tell me a bit about your current level of education and any statistics courses you've taken? This will help me gauge the appropriate difficulty for the problem.
</response>

<teacher> [175c79_teacher]
<tool call>

# Sequential workflow 

In [7]:
# Input arguments are passed to the agent dynamically (e.g. by other agents)
class AddInputArgs(BaseModel):
    a: int = Field(..., description="First number to add.")


# User arguments are passed to the agent statically via run_args in RunContextWrapper
class AddUserArgs(LLMPromptArgs):
    b: int


class AddResponse(BaseModel):
    a_plus_b: int


# The input prompt template is used to combine the user and received arguments
# and format them for the LLM.
add_in_prompt = "Add {a} and {b}. Your only output is the resulting number."


add_agent = LLMAgent[AddInputArgs, AddResponse, None](
    name="add_agent",
    llm=LiteLLM(model_name="gpt-4.1"),
    usr_args_schema=AddUserArgs,
    in_prompt=add_in_prompt,
    # Reset message history to system prompt (if provided) before each run
    reset_memory_on_run=True,
)


@add_agent.parse_output
def parse_output(conversation: Messages, **kwargs) -> AddResponse:
    return AddResponse(a_plus_b=int(str(conversation[-1].content)))

In [8]:
class MultiplyUserArgs(LLMPromptArgs):
    c: int


class MultiplyResponse(BaseModel):
    c_a_plus_b: int


multiply_in_prompt = (
    "Multiply {a_plus_b} by {c}. Your only output is the resulting number."
)

multiply_agent = LLMAgent[AddResponse, MultiplyResponse, None](
    name="multiply_agent",
    llm=LiteLLM(model_name="gpt-4.1"),
    usr_args_schema=MultiplyUserArgs,
    in_prompt=multiply_in_prompt,
    reset_memory_on_run=True,
)


@multiply_agent.parse_output
def parse_output(conversation: Messages, **kwargs) -> MultiplyResponse:
    return MultiplyResponse(c_a_plus_b=int(str(conversation[-1].content)))

In [9]:
seq_agent = SequentialWorkflow[AddInputArgs, MultiplyResponse, None](
    name="seq_agent", subprocs=[add_agent, multiply_agent]
)

In [10]:
add_run_args = RunArgs(usr=AddUserArgs(b=3))
multiply_run_args = RunArgs(usr=MultiplyUserArgs(c=6))

ctx = RunContext[None](
    run_args={"add_agent": add_run_args, "multiply_agent": multiply_run_args},
    print_messages=True,
)

In [11]:
# Can pass batched arguments
out = await seq_agent.run(in_args=[AddInputArgs(a=2)], ctx=ctx)
print_single_output(out)

<add_agent> [21d4c9_seq_agent/add_agent/0]
<input>
Add 2 and 3. Your only output is the resulting number.
</input>

<add_agent> [21d4c9_seq_agent/add_agent/0]
<response>
5
</response>

------------------------------------
I/O/R/C tokens: 24/1/-/-
<multiply_agent> [21d4c9_seq_agent/multiply_agent]
<input>
Multiply 5 by 6. Your only output is the resulting number.
</input>

<multiply_agent> [21d4c9_seq_agent/multiply_agent]
<response>
30
</response>

------------------------------------
I/O/R/C tokens: 25/1/-/-

<final answer>
c_a_plus_b=30
</final answer>


In [14]:
ctx = RunContext[None](
    run_args={"add_agent": add_run_args, "multiply_agent": multiply_run_args},
    print_messages=False,
)

events = []
async for event in print_event_stream(
    seq_agent.run_stream(in_args=[AddInputArgs(a=2), AddInputArgs(a=4)], ctx=ctx)
):
    events.append(event)



<add_agent> [355ad6_seq_agent/add_agent/0]
<input>
Add 2 and 3. Your only output is the resulting number.
</input>

<add_agent> [355ad6_seq_agent/add_agent/1]
<input>
Add 4 and 3. Your only output is the resulting number.
</input>

<add_agent> [355ad6_seq_agent/add_agent/1]
<response>
7
</response>

<add_agent> [355ad6_seq_agent/add_agent/0]
<response>
5
</response>
[AddResponse(a_plus_b=5), AddResponse(a_plus_b=7)]

<add_agent> [355ad6_seq_agent/add_agent]
<processor output>
{
  "a_plus_b": 5
}
{
  "a_plus_b": 7
}
</processor output>

<multiply_agent> [355ad6_seq_agent/multiply_agent/0]
<input>
Multiply 5 by 6. Your only output is the resulting number.
</input>

<multiply_agent> [355ad6_seq_agent/multiply_agent/1]
<input>
Multiply 7 by 6. Your only output is the resulting number.
</input>

<multiply_agent> [355ad6_seq_agent/multiply_agent/0]
<response>
30
</response>

<multiply_agent> [355ad6_seq_agent/multiply_agent/1]
<response>
42
</response>
[MultiplyResponse(c_a_plus_b=30), Mult

# Agents as tools

When agents are used as tools, their `in_args` become the tool inputs.

This is how one can implement a manager + helpers architecture.

In [15]:
seq_tool = seq_agent.as_tool(
    tool_name="seq_agent_tool",
    tool_description=(
        "A sequential agent that adds 3 to a given integer, "
        "then multiplies the result by 5."
    ),
)

The JSON schema of `in_args` is preserved:

In [16]:
seq_tool.in_type.model_json_schema()

{'properties': {'a': {'description': 'First number to add.',
   'title': 'A',
   'type': 'integer'}},
 'required': ['a'],
 'title': 'AddInputArgs',
 'type': 'object'}

In [17]:
await seq_tool(a=15, ctx=ctx)

MultiplyResponse(c_a_plus_b=108)

# Teacher / students

A more advanced example of multi-agent debate, where agents communicate using the actor model.

Communication schemas

In [6]:
# Teacher can choose which students to send the message to
class TeacherExplanation(BaseModel):
    explanation: str
    recipients: list[str]


# Students can only ask questions to the teacher
class StudentQuestion(BaseModel):
    question: str

In [7]:
pool = PacketPool[Any]()

#### Teacher

In [8]:
teacher_sys_prompt = """
You are a teacher explaining quantum gravity to a 2-year old baby (named student1) and a 30-year old graphic designer (named student2). 
Start explaining, while stopping occasionally to let the students ask questions. 
At the very end of every message, you must specify the recipients of your message 
as a list of selected student names with each name in angle brackets, for example: [<Alice>, <Bob>]. 
You should also give give students simple puzzles to test their understanding. 
Do not ask new questions before the students have answered the previous ones. 
When you make sure that the students have understood the topic, you MUST say exactly "Goodbye, students!" and terminate the conversation.
"""

teacher = LLMAgent[StudentQuestion, TeacherExplanation, None](
    name="teacher",
    llm=LiteLLM(model_name="gpt-4o"),
    sys_prompt=teacher_sys_prompt,
    packet_pool=pool,
    # all available recipients to choose from:
    recipients=["student1", "student2"],
)


@teacher.parse_output
def parse_teacher_output(conversation: Messages, **kwargs) -> TeacherExplanation:
    message = str(conversation[-1].content)
    match = re.search(r"\[(.*?)\]", message)
    if match:
        content = match.group(1)
        recipients = re.findall(r"<(.*?)>", content)
    else:
        recipients = []
    explanation = message.split("[")[0].strip()

    # `selected_recipient_ids` is a required field for `DynCommPayload`
    return TeacherExplanation(explanation=explanation, recipients=recipients)


@teacher.set_recipients
def set_teacher_recipients(
    out_packet: Packet[TeacherExplanation], ctx: RunContext[Any]
) -> None:
    out_packet.recipients = out_packet.payloads[0].recipients


@teacher.exit_communication
def teacher_exit_condition(out_packet: Packet[TeacherExplanation], ctx) -> bool:
    # Finish the conversation if the teacher says "Goodbye, students!"
    message = out_packet.payloads[0].explanation

    return "Goodbye, students!" in message

#### Students

In [9]:
student_sys_prompts = [
    """
You are a 4-year old child trying to make sense of physics. 
Your name is <student1>.
Talk to the teacher to understand the topic.
There is also another student in the class, a 30 year old graphic designer. 
You talk to the teacher only.
""",
    """
You are a 30-year old experienced graphic designer curious about physics. 
Your name is <student2>.
Ask questions to the teacher until you understand the topic. 
Attempt to answer the teacher's questions, but if you don't understand,
ask for clarification. 
There is also another student in the class, a 4-year old child.
You talk to the teacher only.
""",
]


def make_student_agent(name: str, sys_prompt: str):
    return LLMAgent[TeacherExplanation, StudentQuestion, None](
        name=name,
        llm=LiteLLM(model_name="gpt-4o"),
        sys_prompt=sys_prompt,
        packet_pool=pool,
        recipients=["teacher"],
    )


student1 = make_student_agent("student1", student_sys_prompts[0])
student2 = make_student_agent("student2", student_sys_prompts[1])


@student1.parse_output
def parse_student1_output(conversation: Messages, **kwargs) -> StudentQuestion:
    return StudentQuestion(question="<student1>: " + str(conversation[-1].content))


@student2.parse_output
def parse_student2_output(conversation: Messages, **kwargs) -> StudentQuestion:
    return StudentQuestion(question="<student2>: " + str(conversation[-1].content))

Specify shared context 

In [10]:
ctx = RunContext[None](print_messages=True)
ctx.printer.color_by = "agent"

Run and wait until completion

In [11]:
teacher.start_listening(ctx=ctx)
student1.start_listening(ctx=ctx)
student2.start_listening(ctx=ctx)

# Teacher starts the conversation by posting a message to the pool
_ = await teacher.run("start", ctx=ctx)

<teacher> [5f0d6f_teacher]
<system>
You are a teacher explaining quantum gravity to a 2-year old baby (named student1) and a 30-year old graphic designer (named student2). 
Start explaining, while stopping occasionally to let the students ask questions. 
At the very end of every message, you must specify the recipients of your message 
as a list of selected student names with each name in angle brackets, for example: [<Alice>, <Bob>]. 
You should also give give students simple puzzles to test their understanding. 
Do not ask new questions before the students have answered the previous ones. 
When you make sure that the students have understood the topic, you MUST say exactly "Goodbye, students!" and terminate the conversation.
</system>

<teacher> [5f0d6f_teacher]
<input>
start
</input>

<teacher> [5f0d6f_teacher]
<response>
Hello, student1 and student2! Today, we're going to talk about something really interesting called "quantum gravity."

Imagine you have a small ball, like one of y

# Custom API providers and HTTP clients

In [29]:
custom_provider = APIProvider(
    name="openrouter",
    base_url="https://openrouter.ai/api/v1",
    api_key=os.getenv("OPENROUTER_API_KEY"),
)

http_client = httpx.AsyncClient(
    timeout=httpx.Timeout(10),
    limits=httpx.Limits(max_connections=10),
)

chatbot = LLMAgent[None, list[int], None](
    name="chatbot",
    llm=OpenAILLM(
        model_name="deepseek/deepseek-r1-0528",
        api_provider=custom_provider,
        async_http_client=http_client,
    ),
)


ctx = RunContext[None](print_messages=True)
out = await chatbot.run(
    "Output a list of 3 integers from 0 to 10 as a python array, no talking",
    ctx=ctx,
)
print_single_output(out)

<chatbot> [e7e14c_chatbot]
<input>
Output a list of 3 integers from 0 to 10 as a python array, no talking
</input>

<chatbot> [e7e14c_chatbot]
<response>
[
  3,
  7,
  9
]
</response>

------------------------------------
I/O/R/C tokens: 26/58/-/-

<final answer>
[3, 7, 9]
</final answer>
